## ACS Housing Units by Building Size for Urban Villages, City of Phoenix

- https://www.census.gov/data/developers/data-sets/acs-5year.html
- https://api.census.gov/data/2013/acs/acs5/variables.html
- https://api.census.gov/data/2021/acs/acs5/variables.html

In [1]:
import pandas as pd
import math
import numpy as np

In [2]:
import getters as get
import utilcalcs as calc
import geo_agg
from acs_bsize_vars import *

In [3]:
#Search parameters - NOTE DIFFERENT VARIABLE # ASSIGNMENTS IN 2010 VS 2020
y1 = '2021'
y0_5 = '2013'
y0_1 = '2010'

cols = 'group(B25024)'

source_1 = 'acs/acs1'
source_5 = 'acs/acs5'

#### Calculate building size columns function

In [4]:
# function to sum select building columns & recalc MOES
def group_bsize(df,year):
    df['u_1_E'] = df.loc[:,U1E].sum(axis=1)
    df['u_1_M'] = df.apply(lambda x: (calc.get_moe(x[U1M])),axis=1)
    df['u_24_E'] = df.loc[:,U24E].sum(axis=1)
    df['u_24_M'] = df.apply(lambda x: (calc.get_moe(x[U24M])),axis=1)
    df['u_520_E'] = df.loc[:,U520E].sum(axis=1)
    df['u_520_M'] = df.apply(lambda x: (calc.get_moe(x[U520M])),axis=1)
    df['u_o20_E'] = df.loc[:,Uo20E].sum(axis=1)
    df['u_o20_M'] = df.apply(lambda x: (calc.get_moe(x[Uo20M])),axis=1)
    df['u_oth_E'] = df.loc[:,UOthE].sum(axis=1)
    df['u_oth_M'] = df.apply(lambda x: (calc.get_moe(x[UOthM])),axis=1)
    df = df.rename(hou_rename,axis=1).drop(columns=hou_vars)
    return df

#### make urban villages from Census Block Groups
def make_uvil(geo_df,year): 
    df = get.get_bgp(source,year,cols)
    df = get.clean_data(df)
    df = group_bsize(df,year)
    
    #df = pd.merge(geo_df,df,how='left',left_on='geoid',right_on='GEO_ID')
    #df = df.drop(['geoid','GEO_ID'],axis=1)
    #df = geo_agg.sumgeo_cv(df,'name')
    return df

In [20]:
# make city and larger geos
def make_table(source,year,cols,geotype='big'):
    if geotype=='big':
        ph = get.get_phx(source,year,cols)
        mar = get.get_maricopa(source,year,cols)
        us = get.get_us(source,year,cols)
        az = get.get_az(source,year,cols)
        cit = get.get_comp_cities(source,year,cols)
        df = pd.concat([ph,mar,us,az,cit])
        df = df.drop(['NAME','us','state'],axis=1)
        aggtype='GEO_ID'
    elif geotype=='uv':
        df = get.get_bgp(source,year,cols)
        aggtype='name'
    elif geotype=='az_places':
        pl = get.get_az_place(source,year,cols)
        aggtype='GEO_ID'
    else:
        pass
    df = get.clean_table(df)
    df = group_bsize(df,year)
    if geotype=='uv': df = geo_agg.make_uv(df,int(year))
    df = geo_agg.sumgeo_cv(df,aggtype)
    
    for col in df.columns[1:]:
        df.rename(columns={col:f'{col[:-1]}{year[-2:]}{col[-1:]}'},inplace=True)
    return df

### Make a table for larger geos

In [6]:
geo_y1_1y = make_table(source_1,y1,cols)
geo_y0_1y = make_table(source_1,y0_1,cols)

In [7]:
geo_y1_1y.head()

,GEO_ID,u_59_21E,u_59_21M,u_59_21C,u_oth_21E,u_oth_21M,u_oth_21C,tot_21E,tot_21M,tot_21C,...,u_o50_21C,u_2049_21E,u_2049_21M,u_2049_21C,u_24_21E,u_24_21M,u_24_21C,u_1_21E,u_1_21M,u_1_21C
0,0455000,32376.0,3355.0,6.299462,21057.0,2722.223356,7.858893,644625.0,7668.0,0.723118,...,4.193097,26726.0,3005.0,6.835096,37380.0,3514.621032,5.715751,419998.0,8596.223182,1.244212
1,04013,80306.0,4870.0,3.686507,85357.0,4740.793710,3.376339,1849103.0,2270.0,0.074627,...,2.521754,54936.0,4446.0,4.919790,85677.0,5308.914955,3.766826,1310255.0,12294.236739,0.570400
2,0100000US,6379448.0,48528.0,0.462427,8174677.0,52223.815429,0.388358,142148050.0,4219.0,0.001804,...,0.319562,5252400.0,42709.0,0.494306,10768173.0,58815.579475,0.332035,96627907.0,128445.324749,0.080807
3,0400000US04,115712.0,6571.0,3.452130,299928.0,8810.548280,1.785747,3138685.0,419.0,0.008115,...,2.234384,79237.0,5909.0,4.533358,141542.0,6788.725948,2.915661,2201550.0,15552.287581,0.429437
4,4865000,47073.0,4509.0,5.822942,9465.0,1758.265054,11.292701,603670.0,6093.0,0.613572,...,5.784510,32766.0,3995.0,7.411864,42606.0,4481.516038,6.394231,380638.0,7373.168790,1.177541


In [10]:
change_vars = ['tot','u_1','u_24','u_59','u_1019','u_2049','u_o50','u_oth','u_520','u_o20']

In [11]:
big = pd.merge(geo_y0_1y,geo_y1_1y,how='left',on='GEO_ID')
big = calc.calc_change(big,y0_1,y1,change_vars)

In [13]:
rename_geos = {'0455000':'Phoenix', '04013':'Maricopa', '0100000US':'US', '0400000US04':'AZ',\
               '0473000':'Tempe','0465000':'Scottsdale', '0427820':'Glendale',\
               '4865000':'San Antonio', '4819000':'Dallas', '1235000':'Jacksonville'}

In [14]:
big['GEO_ID'] = big.GEO_ID.map(rename_geos)

In [15]:
big

,GEO_ID,u_59_10E,u_59_10M,u_59_10C,u_oth_10E,u_oth_10M,u_oth_10C,tot_10E,tot_10M,tot_10C,...,u_520_1021E,u_520_1021M,u_520_1021C,u_520_10P,u_520_21P,u_o20_1021E,u_o20_1021M,u_o20_1021C,u_o20_10P,u_o20_21P
0,Phoenix,36268.0,3832.0,6.422971,18282.0,2429.106832,8.077129,601370.0,7751.0,0.783520,...,-23173.0,7707.301668,20.218745,0.149688,0.103696,31274.0,7649.414357,14.868900,0.113193,0.154113
1,Maricopa,91973.0,5703.0,3.769442,88636.0,5108.048649,3.503313,1640448.0,2235.0,0.082823,...,-43843.0,11331.985616,15.712303,0.124875,0.087074,74453.0,10114.103668,8.258084,0.080681,0.111841
2,US,6338380.0,40171.0,0.385273,8746954.0,49220.814398,0.342079,131791065.0,5741.0,0.002648,...,106323.0,92858.404671,53.091879,0.093096,0.087061,2951544.0,81092.168919,1.670182,0.085364,0.099908
3,AZ,121145.0,6479.0,3.251147,316141.0,8085.711410,1.554789,2846738.0,629.0,0.013432,...,-43863.0,13353.726633,18.507095,0.094380,0.071626,84184.0,12352.339819,8.919772,0.065573,0.086295
4,San Antonio,38422.0,3294.0,5.211680,7665.0,1482.663144,11.758839,525799.0,4754.0,0.549634,...,12811.0,7807.603922,37.048347,0.163093,0.163276,26343.0,6429.786233,14.837659,0.087587,0.119926
5,Dallas,59083.0,4176.0,4.296670,6977.0,1434.084028,12.495107,514372.0,6971.0,0.823857,...,-9258.0,9365.171969,61.493991,0.247842,0.199781,56989.0,9727.580789,10.376429,0.193611,0.264590
6,Jacksonville,20223.0,2600.0,7.815592,18165.0,1864.945308,6.241153,368037.0,4746.0,0.783917,...,5299.0,6857.879701,78.673784,0.134087,0.129401,5258.0,6141.725572,71.007451,0.103014,0.102225


In [16]:
big.to_excel('output/acs_bsize_1yr.xlsx',index=False)

## 5-year for AZ places and urban villages

#### urban villages - 5yr

In [21]:
uv_y1_5y = make_table(source_5,y1,cols,'uv')
uv_y0_5y = make_table(source_5,y0_5,cols,'uv')

In [24]:
uv = pd.merge(uv_y0_5y,uv_y1_5y,how='left',on='name')
uv = calc.calc_change(uv,y0_5,y1,change_vars)

In [25]:
uv

,name,u_59_13E,u_59_13M,u_59_13C,u_oth_13E,u_oth_13M,u_oth_13C,u_24_13E,u_24_13M,u_24_13C,...,u_520_1321E,u_520_1321M,u_520_1321C,u_520_13P,u_520_21P,u_o20_1321E,u_o20_1321M,u_o20_1321C,u_o20_13P,u_o20_21P
0,Laveen,0.0,63.686733,0.000000,483.0,168.863850,21.253167,72.0,106.780148,90.155478,...,48.0,148.801882,188.452231,0.000000,0.002495,18.0,136.663821,461.546171,0.000000,0.000936
1,Deer Valley,4351.0,498.541874,6.965410,3889.0,426.500879,6.666779,2820.0,472.702867,10.189978,...,-1581.0,1308.904122,50.328045,0.148414,0.122337,2960.0,908.699620,18.662195,0.058661,0.097127
2,Maryvale,4282.0,496.469536,7.048230,983.0,281.293441,17.395631,2797.0,392.455093,8.529658,...,-658.0,1085.284755,100.265588,0.139931,0.122493,-225.0,1101.503972,297.603235,0.091755,0.083221
3,North Mountain,6547.0,650.470599,6.039756,1219.0,262.590937,13.095139,5497.0,581.226290,6.427671,...,-3598.0,1397.968526,23.619480,0.204628,0.155396,4071.0,1311.935212,19.590494,0.113672,0.169646
4,Paradise Valley,4496.0,566.107764,7.654325,4231.0,408.387071,5.867635,3468.0,461.150735,8.083472,...,-2578.0,1232.338428,29.059034,0.125381,0.093177,2110.0,1037.923889,29.903165,0.086648,0.115833
5,Encanto,1871.0,328.984802,10.688977,460.0,169.457959,22.394338,2468.0,343.608789,8.463563,...,-855.0,747.338611,53.135577,0.176590,0.139423,1593.0,875.760241,33.419777,0.198798,0.244221
6,Camelback East,5830.0,576.929805,6.015733,1035.0,314.656956,18.481239,6627.0,635.096056,5.825813,...,-2374.0,1412.081796,36.158736,0.217362,0.176874,4223.0,1324.497263,19.066197,0.185068,0.234776
7,South Mountain,1640.0,303.703803,11.257462,2256.0,317.932383,8.567020,2799.0,358.668371,7.789764,...,-570.0,662.078545,70.610414,0.090034,0.069821,441.0,529.358102,72.970122,0.050204,0.056924
8,Alhambra,4468.0,515.014563,7.007134,1236.0,272.686633,13.411566,4579.0,509.547839,6.764698,...,-1755.0,1180.074998,40.875830,0.198474,0.166713,1990.0,918.165562,28.048008,0.128396,0.166321
9,Ahwatukee Foothills,2157.0,366.188476,10.320212,50.0,136.036760,165.394237,990.0,244.374712,15.005662,...,-1331.0,725.016552,33.113414,0.142547,0.102755,2063.0,637.475490,18.784445,0.050929,0.110648


In [ ]:
final = pd.concat([dfY1,phY1,usY1])
final.to_excel(f'output/Hou_bsize.xlsx',index=False)